In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from procedures import (
    create_binance_bot,
    create_binance_bot_spot,
    create_bybit_bot,
    load_exchange_key_secret_passphrase,
)
from pure_funcs import get_template_live_config, ts_to_date, get_daily_from_income
from njit_funcs import round_dynamic
from time import time
import pandas as pd
import matplotlib.pyplot as plt

using numba


In [3]:
plt.rcParams["figure.figsize"] = [21, 13]
pd.set_option("display.precision", 10)

In [4]:
config = get_template_live_config()
config["market_type"] = "futures"
config["user"] = "binance_01"
config["exchange"] = load_exchange_key_secret_passphrase(config["user"])[0]
config["symbol"] = "1000PEPEUSDT"  # dummy symbol

spot_quote = "BTC"

n_days = 30
start_time = (time() - 60 * 60 * 24 * n_days) * 1000
end_time = time() * 1000

if config["exchange"] == "binance":
    if config["market_type"] == "spot":
        raise Exception("implementation work in progress")
        bot = await create_binance_bot_spot(config)
        balances = await bot.private_get(bot.endpoints["balance"])
        ball = []
        for x in balances["balances"]:
            onhand = float(x["free"]) + float(x["locked"])
            if onhand != 0.0:
                ball.append({"coin": x["asset"], "onhand": onhand})
        symbols = [x["coin"] + spot_quote for x in ball]
        print(symbols)

    else:
        bot = await create_binance_bot(config)
        poss = await bot.private_get(bot.endpoints["position"])
        tickers = await bot.public_get(bot.endpoints["ticker"])
        tickersd = {x["symbol"]: float(x["bidPrice"]) for x in tickers}
        balance = bot.position["wallet_balance"]
        possl = []
        for x in poss:
            if float(x["positionAmt"]) != 0.0:
                possl.append(
                    {
                        "symbol": x["symbol"],
                        "psize": float(x["positionAmt"]),
                        "pprice": float(x["entryPrice"]),
                        "side": x["positionSide"].lower(),
                    }
                )
                possl[-1]["last_price"] = tickersd[x["symbol"]]
                if possl[-1]["psize"] > 0:
                    possl[-1]["PA_dist"] = (possl[-1]["pprice"] - tickersd[x["symbol"]]) / tickersd[
                        x["symbol"]
                    ]
                else:
                    possl[-1]["PA_dist"] = (tickersd[x["symbol"]] - possl[-1]["pprice"]) / possl[-1][
                        "pprice"
                    ]

elif config["exchange"] == "bybit":
    bot = await create_bybit_bot(config)
    poss = await bot.private_get(bot.endpoints["position"])
    tickers = await bot.public_get(bot.endpoints["ticker"])
    tickersd = {x["symbol"]: float(x["bid_price"]) for x in tickers["result"]}
    balance = bot.position["wallet_balance"]
    possl = []
    for x in poss["result"]:
        x = x["data"]
        if float(x["size"]) != 0.0:
            possl.append(
                {
                    "symbol": x["symbol"],
                    "psize": float(x["size"]),
                    "pprice": float(x["entry_price"]),
                    "side": "long" if x["side"] == "Buy" else "short",
                }
            )
            possl[-1]["last_price"] = tickersd[x["symbol"]]
            if possl[-1]["psize"] > 0:
                possl[-1]["PA_dist"] = (possl[-1]["pprice"] - tickersd[x["symbol"]]) / tickersd[
                    x["symbol"]
                ]
            else:
                possl[-1]["PA_dist"] = (tickersd[x["symbol"]] - possl[-1]["pprice"]) / possl[-1][
                    "pprice"
                ]


pdf = pd.DataFrame(possl)
pdf.loc[:, "pcost"] = abs(pdf.psize * pdf.pprice)
pdf.loc[:, "wallet_exposure"] = pdf.pcost / balance
symbols = sorted(pdf.symbol.unique())

incomes = []
if config["exchange"] == "bybit":
    for symbol in symbols:
        print(f"fetching income for {symbol}")
        income = await bot.get_all_income(start_time=start_time, symbol=symbol)
        incomes += income
elif config["exchange"] == "binance":
    incomes = await bot.get_all_income(start_time=start_time)
idf, bdf = get_daily_from_income(
    sorted(incomes, key=lambda x: x["timestamp"]),
    bot.position["wallet_balance"],
    start_time=start_time,
    end_time=end_time,
)

not using numba
not using numba
not using numba


ERROR:root:error fetching open orders string indices must be integers, not 'str'
Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/exchanges/binance.py", line 317, in fetch_open_orders
    "order_id": int(e["orderId"]),
                    ~^^^^^^^^^^^
TypeError: string indices must be integers, not 'str'
ERROR:root:error with update open orders 'bool' object is not iterable


linear perpetual
initiating position, open orders, fills, exchange config, order book, and emas...
returned: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'}


Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/passivbot.py", line 364, in update_open_orders
    open_orders = [x for x in open_orders if x["symbol"] == self.symbol]
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: 'bool' object is not iterable
Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/exchanges/binance.py", line 568, in fetch_fills
    "symbol": x["symbol"],
              ~^^^^^^^^^^
TypeError: string indices must be integers, not 'str'
Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/exchanges/binance.py", line 513, in fetch_latest_fills
    "order_id": elm["orderId"],
                ~~~^^^^^^^^^^^
TypeError: string indices must be integers, not 'str'
ERROR:root:error fetching pos or balance 0
Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/exchanges/binance.py", line 348, in fetch_position
    assert all(
 

2024-07-06T04:11:57  {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'} 
error fetching fills string indices must be integers, not 'str'
error fetching latest fills string indices must be integers, not 'str'
returned: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'}
returned: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'}
returned: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'}
2024-07-06T04:11:57  {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'} 


Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/passivbot.py", line 432, in update_position
    assert position is not None
           ^^^^^^^^^^^^^^^^^^^^
AssertionError


2024-07-06T04:11:57  {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'} 
done


ERROR:root:error fetching pos or balance 0
Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/exchanges/binance.py", line 348, in fetch_position
    assert all(
           ^^^^
  File "/Users/manhngodh/development/passivbot/exchanges/binance.py", line 349, in <genexpr>
    key in positions[0] for key in ["symbol", "positionAmt", "entryPrice"]
           ~~~~~~~~~^^^
KeyError: 0
ERROR:root:error with update position 
Traceback (most recent call last):
  File "/Users/manhngodh/development/passivbot/passivbot.py", line 432, in update_position
    assert position is not None
           ^^^^^^^^^^^^^^^^^^^^
AssertionError


returned: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'}
returned: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 116.110.42.21'}


KeyError: 'wallet_balance'

In [ ]:
print(f"current balance {round_dynamic(balance, 5)}")
n_days = (time() * 1000 - idf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
print("n days", n_days)
print(f'abs sum {round_dynamic(idf.income.sum(), 4)} {idf.iloc[-1]["token"]}')
print(f'abs adg {round_dynamic(idf.income.sum() / n_days, 4)} {idf.iloc[-1]["token"]}')
print(f"pct sum {((balance + idf.income.sum()) / balance) - 1:.5f}")
print(f"pct adg {((balance + idf.income.sum()) / balance) ** (1 / n_days) - 1:.5f}")

idf.income.cumsum().plot()

In [ ]:
idf.tail(20)

In [ ]:
bdf

In [ ]:
bdf.gain.plot()

In [ ]:
# check income last x days
x = 3
since = (time() - 60 * 60 * 24 * x) * 1000
idf[idf.timestamp > since].groupby("symbol").income.sum().sort_values()

In [ ]:
pdf.sort_values("PA_dist")